In [3]:
import pandas as pd
import sys

# The log file generated by evaluate_eta.py
LOG_FILE = "eta_eva_airport_rawai.csv"
# Define the new columns we will add
SEGMENT_COLS = [
    'actual_segment_travel_sec', 
    'predicted_segment_travel_sec', 
    'segment_delay_sec', 
    'segment_delay_min'
]

def analyze_segment_delays():
    """
    Reads the evaluation log, calculates segment-by-segment delays,
    and SAVES these new calculations back to the CSV file.
    """
    try:
        df = pd.read_csv(LOG_FILE)
    except FileNotFoundError:
        print(f"Error: Could not find the log file: {LOG_FILE}")
        print("Please run evaluate_eta.py first to generate it.")
        return
    except pd.errors.EmptyDataError:
        print(f"Error: The log file {LOG_FILE} is empty.")
        return

    # Clean up the data
    # 1. Remove empty rows (like ',,,,,,,')
    df = df.dropna(how='all')
    
    # 2. Check if analysis has already been run
    #    If we re-run, drop old analysis columns to recalculate
    for col in SEGMENT_COLS:
        if col in df.columns:
            df = df.drop(columns=[col])
            
    # 3. Convert time columns from strings to datetime objects
    try:
        df['predicted_arrival_time'] = pd.to_datetime(df['predicted_arrival_time'])
        df['actual_arrival_time'] = pd.to_datetime(df['actual_arrival_time'])
    except Exception as e:
        print(f"Error converting date columns: {e}")
        print("Please ensure the CSV format is correct.")
        return

    # 4. Sort by bus, then by time/index to prepare for segment calculations
    df = df.sort_values(by=['licence', 'route', 'target_index'])

    # === Segment-by-Segment Calculation ===
    
    # Use groupby().diff() to calculate the time delta between one row and the next.
    # This isolates the time it took to travel *just this segment*.
    # We group by 'licence' and 'route' to ensure we don't calculate the diff
    # between the end of one bus's run and the start of another's.
    
    # 1. Calculate ACTUAL time (in seconds) to travel the segment
    df['actual_segment_travel_sec'] = df.groupby(
        ['licence', 'route']
    )['actual_arrival_time'].diff().dt.total_seconds()

    # 2. Calculate PREDICTED time (in seconds) to travel the segment
    df['predicted_segment_travel_sec'] = df.groupby(
        ['licence', 'route']
    )['predicted_arrival_time'].diff().dt.total_seconds()

    # 3. Calculate the SEGMENT-SPECIFIC delay
    # (Positive = Slower than predicted, Negative = Faster than predicted)
    df['segment_delay_sec'] = (
        df['actual_segment_travel_sec'] - df['predicted_segment_travel_sec']
    )
    
    df['segment_delay_min'] = df['segment_delay_sec'] / 60.0

    # === Save Results back to CSV ===
    try:
        # Save the *entire* DataFrame, now with new columns, back to the file
        df.to_csv(LOG_FILE, index=False)
        print(f"\nSuccessfully updated '{LOG_FILE}' with new segment analysis columns:")
        print(f"  {', '.join(SEGMENT_COLS)}")
    except Exception as e:
        print(f"\nError: Could not save updated CSV file: {e}")
        print("Displaying results in console instead:")
        # Fallback to printing in console if save fails
        print(df[
            ['licence', 'route', 'target_km_interval'] + SEGMENT_COLS
        ].to_string())
    
    # === Display Summaries in Console ===
    
    print("\n" + "="*60)
    print("Average Segment Delay (Route Calibration Guide)")
    print("="*60)
    print("This shows the average delay for each 1km segment.")
    print("(Positive = Segment is SLOW, Negative = Segment is FAST)\n")

    # Group by route and km interval to find the average delay for each
    # This is the most important data for calibration.
    avg_delays = df.groupby(
        ['route', 'target_km_interval']
    )['segment_delay_min'].mean().sort_values(ascending=False)

    print(avg_delays.to_string())

    # ---
    
    print("\n" + "="*60)
    print("Actionable Insights: Worst Performing Segments")
    print("="*60)
    print(">>> TO FIX: Find these 'km_interval' in your '..._speeds.csv' file")
    print(">>> and LOWER their 'avg_speed' value.\n")
    print(avg_delays.head(10).to_string())

    print("\n" + "="*60)
    print("Actionable Insights: Best Performing Segments")
    print("="*60)
    print(">>> TO FIX: Find these 'km_interval' in your '..._speeds.csv' file")
    print(">>> and INCREASE their 'avg_speed' value.\n")
    print(avg_delays.tail(5).to_string())


if __name__ == "__main__":
    analyze_segment_delays()


Successfully updated 'eta_eva_airport_rawai.csv' with new segment analysis columns:
  actual_segment_travel_sec, predicted_segment_travel_sec, segment_delay_sec, segment_delay_min

Average Segment Delay (Route Calibration Guide)
This shows the average delay for each 1km segment.
(Positive = Segment is SLOW, Negative = Segment is FAST)

route             target_km_interval
Airport -> Rawai  48                    1.094455
                  46                    1.036922
                  54                    1.032712
                  53                    0.566235
                  52                    0.089000
                  45                    0.036344
                  44                   -0.001283
                  50                   -0.446252
                  51                   -0.486397
                  47                   -0.856324
                  49                   -0.973042
                  43                         NaN

Actionable Insights: Worst Performi